### Forward kinematics, inverse kinematics from resultant, then check end effector position to verify

Implements joint angle bounds

In [1]:
import numpy as np
from sympy import symbols, Matrix, cos, sin, eye, pi, lambdify
from scipy.optimize import minimize

# Define Symbols
t1, t2, t3, t4, t5, t6, t7, ti = symbols("\\theta_1 \\theta_2 \\theta_3 \\theta_4 \\theta_5 \\theta_6 \\theta_7 \\theta_i")
d1, d2, d3, d4, d5, d6, d7, di = symbols("d_1 d_2 d_3 d_4 d_5 d_6 d_7 d_i")
a1, a2, a3, a4, a5, a6, a7, ai = symbols("a_1 a_2 a_3 a_4 a_5 a_6 a_7 a_i")
al1, al2, al3, al4, al5, al6, al7, ali = symbols("\\alpha_1 \\alpha_2 \\alpha_3 \\alpha_4 \\alpha_5 \\alpha_6 \\alpha_7 \\alpha_i")

# Define General Transformation Matrix
Rx = Matrix([[1, 0, 0, 0],
             [0, cos(ali), -sin(ali), 0],
             [0, sin(ali), cos(ali), 0],
             [0, 0, 0, 1]])

Dx = Matrix([[1, 0, 0, ai],
             [0, 1, 0, 0],
             [0, 0, 1, 0],
             [0, 0, 0, 1]])

Rz = Matrix([[cos(ti), -sin(ti), 0, 0],
             [sin(ti), cos(ti), 0, 0],
             [0, 0, 1, 0],
             [0, 0, 0, 1]])

Dz = Matrix([[1, 0, 0, 0],
             [0, 1, 0, 0],
             [0, 0, 1, di],
             [0, 0, 0, 1]])

T = Dx * Rx * Dz * Rz

L0 = 270.35e-3
L1 = 69e-3
L2 = 364.35e-3
L3 = 69e-3
L4 = 374.29e-3
L5 = 10e-3
L6 = 229.525e-3
L7 = 10e-3

# Define D-H parameters
alpha = Matrix([0,-pi/2, pi/2, -pi/2, pi/2, -pi/2, pi/2])  # radians
theta_left = Matrix([t1, t2+pi/2, t3, t4, t5, t6, t7])  # radians
theta_right = Matrix([t1+pi, t2-pi/2, t3, t4, t5, t6, t7])  # radians
a = Matrix([0,L1,0,L3,0,L5,0])  # m
d = Matrix([L0,0,L2,0,L4,0,L6+L7])  # m
assert len(alpha) == len(theta_left) == len(theta_right) == len(a) == len(d)

# Left arm forward kinematics
T_left = []
Tleft_end = eye(4)
for i in range(len(alpha)):
    T_left.append(T.subs({ali: alpha[i], ai: a[i], di: d[i], ti: theta_left[i]}))
    Tleft_end *= T_left[i] # transformation matrix from base frame to end effector

# Right arm forward kinematics
T_right = []
Tright_end = eye(4)
for i in range(len(alpha)):
    T_right.append(T.subs({ali: alpha[i], ai: a[i], di: d[i], ti: theta_right[i]}))
    Tright_end *= T_right[i] # transformation matrix from base frame to end effector

# convert to numpy equations
forward_kinematics_x_left = lambdify((t1, t2, t3, t4, t5, t6, t7), Tleft_end[0,3], 'numpy') # px
forward_kinematics_y_left = lambdify((t1, t2, t3, t4, t5, t6, t7), Tleft_end[1,3], 'numpy') # py
forward_kinematics_z_left = lambdify((t1, t2, t3, t4, t5, t6, t7), Tleft_end[2,3], 'numpy') # pz
forward_kinematics_x_right = lambdify((t1, t2, t3, t4, t5, t6, t7), Tright_end[0,3], 'numpy')
forward_kinematics_y_right = lambdify((t1, t2, t3, t4, t5, t6, t7), Tright_end[1,3], 'numpy')
forward_kinematics_z_right = lambdify((t1, t2, t3, t4, t5, t6, t7), Tright_end[2,3], 'numpy')

# Define the joint limits (radians)
joint_limits = [(-1.7016, 1.7016), (-2.147, 1.047), (-3.0541, 3.0541), (-0.05, 2.618), (-3.059, 3.059), (-1.5707, 2.094), (-3.059, 3.059)]

# Optimization
def inverse_kinematics(initial_guess, goal, arm='left'):
    result = minimize(objective_function, initial_guess, args=(goal, arm), bounds=joint_limits)
    return result.x

# Modified objective function to penalize solutions outside joint limits
def objective_function(thetas, goal, arm='left'):
    if arm == 'left':
        actual_x = forward_kinematics_x_left(*thetas)
        actual_y = forward_kinematics_y_left(*thetas)
        actual_z = forward_kinematics_z_left(*thetas)
    elif arm == 'right':
        actual_x = forward_kinematics_x_right(*thetas)
        actual_y = forward_kinematics_y_right(*thetas)
        actual_z = forward_kinematics_z_right(*thetas)
    else:
        raise ValueError("Invalid arm specified. Use 'left' or 'right'.")
    
    # Calculate error
    error = np.sum((np.array([actual_x, actual_y, actual_z]) - goal)**2)
    
    # Penalize solutions outside joint limits
    for theta, (lower, upper) in zip(thetas, joint_limits):
        if theta < lower or theta > upper:
            error += 1000  # Penalty for violating joint limits
    
    return error

# Example usage
initial_guess = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])  # Initial guess for joint angles
goal_position = np.array([0.75, 0.3, 0.3])
arm_choice = 'left'  # 'left' or 'right'
optimized_angles = inverse_kinematics(initial_guess, goal_position, arm=arm_choice)
optimized_angles_degrees = np.degrees(optimized_angles)
optimized_angles_rounded = [round(angle, 4) for angle in optimized_angles_degrees]
print("Optimized Joint Angles (in degrees):", optimized_angles_rounded)

Optimized Joint Angles (in degrees): [22.8756, 11.487, 3.127, -2.8648, 1.3891, -89.9945, 0.0]


### Forward Kinematics (check inv-kin result)

In [2]:
# Initialize T_end
T_end = eye(4)

# Loop through thetas and calculate positions
for i in range(len(optimized_angles)):
    T_end *= T_left[i]

# Substitute theta values into T_end and then extract positions
x_pos = T_end[0, 3].subs({t1: optimized_angles[0], t2: optimized_angles[1], t3: optimized_angles[2], 
                          t4: optimized_angles[3], t5: optimized_angles[4], t6: optimized_angles[5], 
                          t7: optimized_angles[6]})
y_pos = T_end[1, 3].subs({t1: optimized_angles[0], t2: optimized_angles[1], t3: optimized_angles[2], 
                          t4: optimized_angles[3], t5: optimized_angles[4], t6: optimized_angles[5], 
                          t7: optimized_angles[6]})
z_pos = T_end[2, 3].subs({t1: optimized_angles[0], t2: optimized_angles[1], t3: optimized_angles[2], 
                          t4: optimized_angles[3], t5: optimized_angles[4], t6: optimized_angles[5], 
                          t7: optimized_angles[6]})

In [3]:
# Calculate end effector position
if arm_choice == 'left':
    x_pos_val = forward_kinematics_x_left(*optimized_angles)
    y_pos_val = forward_kinematics_y_left(*optimized_angles)
    z_pos_val = forward_kinematics_z_left(*optimized_angles)
elif arm_choice == 'right':
    x_pos_val = forward_kinematics_x_right(*optimized_angles)
    y_pos_val = forward_kinematics_y_right(*optimized_angles)
    z_pos_val = forward_kinematics_z_right(*optimized_angles)
else:
    raise ValueError("Invalid arm specified. Use 'left' or 'right'.")

# Evaluate the expressions to obtain numerical values
x_pos_val = float(x_pos_val)
y_pos_val = float(y_pos_val)
z_pos_val = float(z_pos_val)

x_pos_val, y_pos_val, z_pos_val

(0.7584018315590324, 0.30336059929752796, 0.30036323641324186)